In [1]:
!python -V

Python 3.9.12


In [ ]:
# To activate this environment on your terminal 

 #conda activate mlflow

In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import mlflow

from sklearn.metrics import mean_squared_error

#### Make sure you launch the mlflow UI from the same directory as the scripts/jupyter notebook that is running the experiments (same directory that has the mlflow directory and the database that stores the experiments).

In [ ]:
## run on the terminal 
# cd week02
#mlflow ui --backend-store-uri sqlite:///mlflow.db --port 5002

In [4]:

mlflow.set_tracking_uri("sqlite:///mlflow.db")


In [5]:
mlflow.set_experiment("nyc-taxi-experiment")

2024/06/21 20:08:44 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/my_mlops_bootcamp/week-02/mlruns/1', creation_time=1719000524163, experiment_id='1', last_update_time=1719000524163, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## mlops in the ml server 
![exp_tracked](https://github.com/sheyijojo/my_mlops_bootcamp/blob/main/_assets/mlops_exp.png?raw=true)


In [8]:
mlflow 

# mlflow ui --backend-store-uri sqlite:///mlflow.db

<module 'mlflow' from '/home/codespace/anaconda3/envs/mlflow/lib/python3.9/site-packages/mlflow/__init__.py'>

In [ ]:

# preprocessing stage, run this om the terminal 
# python3 preprocess_data.py --raw_data_path TAXI_DATA_FOLDER/ --dest_path ./output

In [15]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [11]:
## training data
df_train = pd.read_parquet("./TAXI_DATA_FOLDER/green_tripdata_2023-01.parquet")
df_val = pd.read_parquet("./TAXI_DATA_FOLDER/green_tripdata_2023-02.parquet")

In [12]:
len(df_train), len(df_val)

(68211, 64809)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [16]:
target = 'duration'
#y_train = df_train[target].values




'duration'